#Тема «Создание признакового пространства»

##Продолжим обработку данных с Твиттера. 

1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform().

 Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
*	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
*	Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
*	Исключим стоп-слова с помощью stop_words='english'. 
*	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().

2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform().

 Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
*	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
*	Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
*	Исключим стоп-слова с помощью stop_words='english'.
*	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().

3. Проверьте ваши векторайзеры на корпусе который использовали на вебинаре, составьте таблицу метод векторизации и скор который вы получили (в методах векторизации по изменяйте параметры что бы добиться лучшего скора) обратите внимание как падает/растёт скор при уменьшении количества фичей, и изменении параметров, так же попробуйте применить к векторайзерам PCA для сокращения размерности посмотрите на качество сделайте выводы


In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
import pickle
from sklearn import model_selection, preprocessing, linear_model
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

In [2]:
#Загрузка файла
with open('/content/drive/MyDrive/NLP/Less_01/df_processed.pkl', 'rb') as f:
    df = pickle.load(f)
df

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunct, selfish, drag, kid, dysfunc..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thank, lyft, credit, use, caus, offer, wheelc..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesti]"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguid, societi, motiv]"
...,...,...,...,...,...,...,...,...
49154,49155,NaN,thought factory: left-right polarisation! #tru...,thought factory leftright polarisation trump u...,"[thought, factory, leftright, polarisation, tr...","[thought, factory, leftright, polarisation, tr...","[thought, factori, leftright, polaris, trump, ...","[thought, factori, leftright, polaris, trump, ..."
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...,feeling like mermaid hairflip neverready forma...,"[feeling, like, mermaid, hairflip, neverready,...","[feeling, like, mermaid, hairflip, neverready,...","[feel, like, mermaid, hairflip, neverreadi, fo...","[feel, like, mermaid, hairflip, neverreadi, fo..."
49156,49157,NaN,#hillary #campaigned today in #ohio((omg)) &am...,hillary campaigned today in ohioomg amp used w...,"[hillary, campaigned, today, in, ohioomg, amp,...","[hillary, campaigned, today, ohioomg, amp, use...","[hillari, campaign, today, ohioomg, amp, use, ...","[hillari, campaign, today, ohioomg, amp, use, ..."
49157,49158,NaN,"happy, at work conference: right mindset leads...",happy at work conference right mindset leads t...,"[happy, at, work, conference, right, mindset, ...","[happy, work, conference, right, mindset, lead...","[happi, work, confer, right, mindset, lead, cu...","[happi, work, confer, right, mindset, lead, cu..."


In [3]:
df = df.dropna()
df

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunct, selfish, drag, kid, dysfunc..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thank, lyft, credit, use, caus, offer, wheelc..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesti]"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguid, societi, motiv]"
...,...,...,...,...,...,...,...,...
31957,31958,0.0,ate @user isz that youuu?ðððððð...,ate isz that youuu,"[ate, isz, that, youuu]","[ate, isz, youuu]","[ate, isz, youuu]","[ate, isz, youuu]"
31958,31959,0.0,to see nina turner on the airwaves trying to...,to see nina turner on the airwaves trying to w...,"[to, see, nina, turner, on, the, airwaves, try...","[see, nina, turner, airwaves, trying, wrap, ma...","[see, nina, turner, airwav, tri, wrap, mantl, ...","[see, nina, turner, airwav, tri, wrap, mantl, ..."
31959,31960,0.0,listening to sad songs on a monday morning otw...,listening to sad songs on monday morning otw t...,"[listening, to, sad, songs, on, monday, mornin...","[listening, sad, songs, monday, morning, otw, ...","[listen, sad, song, monday, morn, otw, work, sad]","[listen, sad, song, monday, morn, otw, work, sad]"
31960,31961,1.0,"@user #sikh #temple vandalised in in #calgary,...",sikh temple vandalised in in calgary wso conde...,"[sikh, temple, vandalised, in, in, calgary, ws...","[sikh, temple, vandalised, calgary, wso, conde...","[sikh, templ, vandalis, calgari, wso, condemn,...","[sikh, templ, vandalis, calgari, wso, condemn,..."


##Count vectorizer

In [4]:
count_vectorizer = CountVectorizer(max_df=.9, max_features=1000, stop_words='english')
values = count_vectorizer.fit_transform(df['tweet_stemmed'].astype('str'))

feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(values.toarray(), columns = feature_names)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,year,yesterday,yo,yoga,york,young,youth,youtub,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31957,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31958,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31959,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31960,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
count_vectorizer = CountVectorizer(max_df=.9, max_features=1000, stop_words='english')
values = count_vectorizer.fit_transform(df['tweet_lemmatized'].astype('str'))

feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(values.toarray(), columns = feature_names)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,year,yesterday,yo,yoga,york,young,youth,youtub,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31957,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31958,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31959,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31960,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##TF IDF vectorizer

In [6]:
tfidf_vectorizer = TfidfVectorizer(max_df=.9, max_features=1000, stop_words='english')
values = tfidf_vectorizer.fit_transform(df['tweet_stemmed'].astype('str'))

feature_names = tfidf_vectorizer.get_feature_names()
pd.DataFrame(values.toarray(), columns = feature_names)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,year,yesterday,yo,yoga,york,young,youth,youtub,yr,yummi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31960,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
tfidf_vectorizer = TfidfVectorizer(max_df=.9, max_features=1000, stop_words='english')
values = tfidf_vectorizer.fit_transform(df['tweet_lemmatized'].astype('str'))

feature_names = tfidf_vectorizer.get_feature_names()
pd.DataFrame(values.toarray(), columns = feature_names)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,year,yesterday,yo,yoga,york,young,youth,youtub,yr,yummi
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31957,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31958,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31959,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31960,0.0,0.0,0.0,0.0,0.649536,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
df.columns

Index(['id', 'label', 'tweet', 'clean_tweet', 'tweet_token',
       'tweet_token_filtered', 'tweet_stemmed', 'tweet_lemmatized'],
      dtype='object')

In [14]:
df['label'].value_counts()

0.0    29720
1.0     2242
Name: label, dtype: int64

#Учитывая такой перекос моделька скоре всего будет переобучаться.

In [31]:
col = ['tweet_stemmed', 'tweet_lemmatized']
print('CountVectorizer')
print('-' *50)
for i in col:
  train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df[i], df['label'])
  count_vectorizer = CountVectorizer(max_df=.9, max_features=1000, stop_words='english')
  xtrain_count =  count_vectorizer.fit_transform(train_x.astype('str'))
  xvalid_count =  count_vectorizer.fit_transform(valid_x.astype('str'))
  classifier = linear_model.LogisticRegression()
  classifier.fit(xtrain_count, train_y)
  predictions = classifier.predict(xvalid_count)
  acc = accuracy_score(valid_y, predictions)
  print(i, ' - ', acc)
print('-' *50)
print('TfidfVectorizer')
print('-' *50)
for i in col:
  train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df[i], df['label'])
  tfidf_vectorizer = TfidfVectorizer(max_df=.9, max_features=1000, stop_words='english')
  xtrain_tfidf = tfidf_vectorizer.fit_transform(train_x.astype('str'))
  xvalid_tfidf = tfidf_vectorizer.fit_transform(valid_x.astype('str'))
  classifier = linear_model.LogisticRegression()
  classifier.fit(xtrain_tfidf, train_y)
  predictions = classifier.predict(xvalid_tfidf)
  acc = accuracy_score(valid_y, predictions)
  print(i, ' - ', acc)

CountVectorizer
--------------------------------------------------
tweet_stemmed  -  0.9169065198348142
tweet_lemmatized  -  0.9017644850456764
--------------------------------------------------
TfidfVectorizer
--------------------------------------------------
tweet_stemmed  -  0.9256663746715055
tweet_lemmatized  -  0.9240395444875485


#Подкручивать параметры скорее всего не получиться потому что показатели уже на уровне переобучения (скорее запоминания)

* 0.0  -  29720
* 1.0   -  2242
